<a href="https://colab.research.google.com/github/erendagasan/Eren-Dagasan-Personal/blob/main/Stcok_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!pip install -q bta-lib
!pip install -q ta
!pip install -q yfinance

In [28]:
import yfinance as yf
import btalib
import numpy as np
import pandas as pd
from ta.trend import PSARIndicator
from ta.momentum import WilliamsRIndicator
from ta.trend import AroonIndicator
from ta.volume import VolumePriceTrendIndicator
from ta.trend import CCIIndicator
from ta.momentum import ROCIndicator
from ta.trend import ADXIndicator

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_rows', 500)

In [29]:
#@title Stock Lists
xu030 = [
        'AKBNK.IS',
        'AKSEN.IS',
        'ALARK.IS',
        'ARCLK.IS',
        'ASELS.IS',
        'BIMAS.IS',
        'EKGYO.IS',
        'ENKAI.IS',
        'EREGL.IS',
        'FROTO.IS',
        'GARAN.IS',
        'GUBRF.IS',
        'HEKTS.IS',
        'KRDMD.IS',
        'KCHOL.IS',
        'KOZAL.IS',
        'KOZAA.IS',
        'ODAS.IS',
        'PGSUS.IS',
        'SAHOL.IS',
        'SASA.IS',
        'PETKM.IS',
        'SISE.IS',
        'TAVHL.IS',
        'THYAO.IS',
        'TUPRS.IS',
        'TOASO.IS',
        'TCELL.IS',
        'ISCTR.IS',
        'YKBNK.IS'
          ]

xu100 = ['ADESE.IS',
        'AGHOL.IS',
        'AKBNK.IS',
        'AKSA.IS',
        'AKSEN.IS',
        'ALGYO.IS',
        'ALARK.IS',
        'ALKIM.IS',
        'AEFES.IS',
        'ARCLK.IS',
        'ARDYZ.IS',
        'ASELS.IS',
        'AYDEM.IS',
        'BERA.IS',
        'BIMAS.IS',
        'BRYAT.IS',
        'BRISA.IS',
        'CCOLA.IS',
        'CANTE.IS',
        'CEMAS.IS',
        'CIMSA.IS',
        'DOHOL.IS',
        'DOAS.IS',
        'EGEEN.IS',
        'ECILC.IS',
        'EKGYO.IS',
        'ENJSA.IS',
        'ENKAI.IS',
        'ERBOS.IS',
        'EREGL.IS',
        'FROTO.IS',
        'GENIL.IS',
        'GESAN.IS',
        'GLYHO.IS',
        'GOZDE.IS',
        'GUBRF.IS',
        'SAHOL.IS',
        'HEKTS.IS',
        'INDES.IS',
        'IPEKE.IS',
        'ISDMR.IS',
        'ISFIN.IS',
        'ISGYO.IS',
        'ISMEN.IS',
        'IZMDC.IS',
        'KRDMD.IS',
        'KARSN.IS',
        'KARTN.IS',
        'KERVT.IS',
        'KRVGD.IS',
        'KCHOL.IS',
        'KORDS.IS',
        'KOZAL.IS',
        'KOZAA.IS',
        'LOGO.IS',
        'MAVI.IS',
        'MGROS.IS',
        'NTHOL.IS',
        'ODAS.IS',
        'OTKAR.IS',
        'OYAKC.IS',
        'PARSN.IS',
        'PGSUS.IS',
        'PETKM.IS',
        'QUAGR.IS',
        'RTALB.IS',
        'SARKY.IS',
        'SASA.IS',
        'SELEC.IS',
        'SKBNK.IS',
        'SOKM.IS',
        'TAVHL.IS',
        'TKFEN.IS',
        'TKNSA.IS',
        'TOASO.IS',
        'TRGYO.IS',
        'TCELL.IS',
        'TUPRS.IS',
        'THYAO.IS',
        'TTKOM.IS',
        'TTRAK.IS',
        'GARAN.IS',
        'HALKB.IS',
        'ISCTR.IS',
        'TSKB.IS',
        'TURSG.IS',
        'SISE.IS',
        'VAKBN.IS',
        'ULKER.IS',
        'VERUS.IS',
        'VESBE.IS',
        'VESTL.IS',
        'YKBNK.IS',
        'YATAS.IS',
        'ZRGYO.IS',
        'ZOREN.IS']

url = 'https://en.m.wikipedia.org/wiki/Nasdaq-100'
nasdaq100 = pd.read_html(url, attrs={'id': "constituents"})[0]
nasdaq100 = nasdaq100["Ticker"].to_list()

#Functions

In [30]:
def create_indicators(data):
  # Below 30 is oversold, upper 70 is overbought
  data["RSI"] = btalib.rsi(data["Close"], period=14).df

  #If little period conflicts with the bigger one it means buy else sell.
  data["SMA9"] = btalib.sma(data['Close'], period=9).df
  data["SMA20"] = btalib.sma(data['Close'], period=20).df

  #A reading below 20 generally represents an oversold market and a reading above 80 an overbought market. Look only STOCH-D.
  data["STOCH-K"] = btalib.stoch(data['High'], data['Low'], data['Close']).df["k"]
  data["STOCH-D"] = btalib.stoch(data['High'], data['Low'], data['Close']).df["d"]

  #When the MACD line crosses above the signal line, it generates a buy signal. This indicates a potential bullish trend reversal.
  #Additionally, monitor the MACD histogram. When the histogram bars turn positive (change from negative to positive), it confirms the buy signal and suggests increasing buying pressure.
  data["MACD"] = btalib.macd(data['Close']).df["macd"]
  data["SIGNAL"] = btalib.macd(data['Close']).df["signal"]
  data["HISTOGRAM"] = btalib.macd(data['Close']).df["histogram"]

  #Buy Signal: When the price of the stock touches or crosses below the lower Bollinger Band, it can be considered a buy signal.
  #This suggests that the stock may be oversold, and a potential price rebound is expected.
  #Sell Signal: When the price of the stock touches or crosses above the upper Bollinger Band, it can be considered a sell signal.
  #This indicates that the stock may be overbought, and a potential price correction is expected.
  data["BB-UPPER"] = btalib.bbands(data['Close']).df['top']
  data["BB-MID"] = btalib.bbands(data['Close']).df['mid']
  data["BB-LOWER"] = btalib.bbands(data['Close']).df['bot']

  #When the current price of the stock falls below a certain number of standard deviations (GET THE MIN AND MAX VALUE OF THE COLUMN AND MAKE IT THE THRESHOLD)
  #from the mean, it generates a buy signal. This suggests that the price has deviated significantly from the average and may present a buying opportunity.
  data["STDEV"] = data["Close"].rolling(window=10).std()

  #Generate a buy signal when the price crosses above the PSAR.
  #This indicates a potential reversal in the downward trend and suggests a buying opportunity.
  data["PSAR"] = PSARIndicator(data["High"], data["Low"], data["Close"]).psar()

  #Williams
  #Use the Williams Percent Range (%R) formula to calculate the %R values based on historical price data.
  #The %R values range from -100 to 0, where values close to -100 indicate oversold conditions and values close to 0 indicate overbought conditions.
  #Generate a buy signal when the %R value crosses above a certain threshold (e.g., -80) from below.
  #This indicates a potential reversal from oversold conditions and suggests a buying opportunity.
  data["WILLIAMS"] = WilliamsRIndicator(data["High"], data["Low"], data["Close"]).williams_r()

  #The difference between these two lines indicates whether there is overbought (a positive number) or oversold (a negative number).
  data["AROON"] = AroonIndicator(close=data["Close"], window=25).aroon_indicator()

  #obv
  data["OBV"] = VolumePriceTrendIndicator(close=data['Close'], volume=data['Volume']).volume_price_trend()

  #CCI
  data['CCI'] = CCIIndicator(close=data['Close'], low=data["Low"], high=data["High"], window=14).cci()

  #ROC
  data['ROC'] = ROCIndicator(close=data['Close'], window=5).roc()

  #BULLBEARPOWER
  data['BULL'] = data['High'] - (data['High'].rolling(13).max() + data['Low'].rolling(13).min()) / 2
  data['BEAR'] = data['Low'] - (data['High'].rolling(13).max() + data['Low'].rolling(13).min()) / 2

  #ADX
  adx_indicator = ADXIndicator(high=data['High'], low=data['Low'], close=data['Close'], window=14)
  data['ADX'] = adx_indicator.adx()
  data['+DI'] = adx_indicator.adx_pos()
  data['-DI'] = adx_indicator.adx_neg()

  data = data.dropna()
  data = data.reset_index()
  return data

def score(data):
  scores = []

  for row in range(data.shape[0]):
    #MAX 15.25
    #MIN -15.25

    min_score = -15.25
    max_score = 15.25
    score = 0

    if data["RSI"][row] < 30:
      score += 1
    elif data["RSI"][row] > 70:
      score+=-1
    # else:
    #   score+=(100-int(data["RSI"][row]))/70

    if data["SMA9"][row] > data["SMA20"][row] and data["Close"][row] > data["SMA9"][row]:
      score += 1
    else:
      score += -1

    if data["STOCH-D"][row] <= 20:
      score += 1
    elif data["STOCH-D"][row] >= 80:
      score += -1

    if data["MACD"][row] > data["SIGNAL"][row]:
      score+=1
    if row > 0 and data["MACD"][row] > data["SIGNAL"][row] and data["HISTOGRAM"][row-1] < 0 and data["HISTOGRAM"][row] > 0:
      score+=1
    if row > 0 and data["MACD"][row] < data["SIGNAL"][row] and data["HISTOGRAM"][row-1] > 0 and data["HISTOGRAM"][row] > 0:
      score+=-1
    elif data["MACD"][row] < data["SIGNAL"][row]:
      score+=-1

    if data["BB-LOWER"].iloc[row] > data["Close"].iloc[row]:
        score += 1
    elif data["BB-MID"].iloc[row] < data["Close"].iloc[row] and data["BB-LOWER"].iloc[row] - data["Close"].iloc[row] < data["BB-MID"].iloc[row] - data["Close"].iloc[row]:
        score += 0.25
    elif data["BB-MID"].iloc[row] > data["Close"].iloc[row] and data["BB-LOWER"].iloc[row] - data["Close"].iloc[row] > data["BB-MID"].iloc[row] - data["Close"].iloc[row]:
        score -= 0.25
    elif data["BB-UPPER"].iloc[row] < data["Close"].iloc[row]:
        score -= 1

    if (data["STDEV"].max() + data["STDEV"].min()) / 2 > data["STDEV"][row]:
      score+=1
    else:
      score+=-1

    if data["PSAR"][row] < data["Close"][row]:
      score+=1
    else:
      score+=-1

    if data["WILLIAMS"][row] < -80:
      score+=1
    elif data["WILLIAMS"][row] > -20:
      score+=-1

    if data["AROON"][row] > 0:
      score+=-1
    else:
      score+=1

    if row > 0 and data["OBV"][row-1] < data["OBV"][row] and data["Close"][row] > data["Close"][row-1]:
      score+=1
    elif row > 0 and data["OBV"][row-1] > data["OBV"][row] and data["Close"][row] < data["Close"][row-1]:
      score+=-1

    if row > 0 and (data['CCI'][row] > -100) & (data['CCI'][row-1] <= -100):
      score+=1
    elif row > 0 and (data['CCI'][row] < 100) & (data['CCI'][row-1] >= 100):
      score-=1

    if data["ROC"][row] > 0:
      score+=1
    elif data["ROC"][row] < 0:
      score-=1

    if data["BULL"][row] > 0:
      score+=1
    elif data["BEAR"][row] < 0:
      score-=1

    if (data['ADX'][row] > 25) and (data['+DI'][row] > data['-DI'][row]):
      score+=1
    elif (data['ADX'][row] > 50) and (data['+DI'][row] < data['-DI'][row]):
      score-=1

    #normalized score between 0-100
    normalized_score = ((score - min_score) / (max_score - min_score)) * (100 - 0) + 0
    scores.append(normalized_score)

  data["SCORE"] = scores
  return data

def save_to_excel(stock, data):
  output_file = stock + ":" + start + ":" + end + '.xlsx'
  writer = pd.ExcelWriter(output_file)
  data.to_excel(writer, index=True, sheet_name=stock)
  writer.save()

def buy_or_sell(stock, data, balance, upper_score, lower_score):
  stock_balance = 0
  transactions = []

  for row in range(data.shape[0]):
    if row > 0 and data["SCORE"].iloc[row] >= upper_score and data["SCORE"].iloc[row-1] < upper_score and balance > data["Close"].iloc[row]:
      stock_balance = balance / data["Close"].iloc[row]
      balance-= stock_balance*data["Close"].iloc[row]
      transactions.append([stock, "BUY", data["Date"].iloc[row], data["Close"].iloc[row], stock_balance, balance])

    elif row > 0 and data["SCORE"].iloc[row] <= lower_score and data["SCORE"].iloc[row-1] > lower_score and stock_balance > 1:
      balance+=stock_balance*data["Close"].iloc[row]
      stock_balance-=stock_balance
      transactions.append([stock,"SELL", data["Date"].iloc[row], data["Close"].iloc[row], stock_balance, balance])

  transactions_df = pd.DataFrame(transactions, columns=["STOCK","B/S", "DATE", "PRICE", "STOCK BALANCE", "TRY BALANCE"])
  transactions_df["LAST PRICE"] = data["Close"].iloc[-1]
  return transactions_df

def test_the_market(market, buy_threshold, sell_threshold, start, end, balance):
  test_df = pd.DataFrame()

  for stock in market:
    try:
      data = yf.download(stock, start, end, progress=False)
      data = create_indicators(data)
      data = score(data)
      simulation_df = buy_or_sell(stock, data, balance=balance, upper_score=buy_threshold, lower_score=sell_threshold)
      test_df = pd.concat([test_df, simulation_df])

    except:
      print("Data error with stock named: " + stock)

  test_df = test_df.reset_index().drop(["index"], axis=1)

  for row in range(test_df.shape[0]):
    if test_df["B/S"].iloc[row] == "BUY" and test_df["B/S"].iloc[row+1] == "BUY" and not test_df["B/S"].iloc[row] == "CLOSE" and not test_df["B/S"].iloc[row+1] == "CLOSE":
      add = pd.DataFrame({"STOCK": [test_df["STOCK"][row]], "B/S": ["CLOSE"], "DATE": ["TODAY"], "PRICE": [test_df["LAST PRICE"][row]],
                          'STOCK BALANCE': ["0"], 'TRY BALANCE': [test_df["STOCK BALANCE"][row]*test_df["LAST PRICE"][row]], 'LAST PRICE' :[test_df["LAST PRICE"][row]]})

      df_top = test_df.loc[:row]
      df_bottom = test_df.loc[row+1:]
      test_df = pd.concat([df_top, add, df_bottom], ignore_index=True)

  return test_df

def optimal_threshold(market, start, end):
  tries = []

  for buy_threshold in range(40, 80):
    print(f"Tried buy threshold is: {buy_threshold}")
    for sell_threshold in range(40, 80):
      print(f"Tried sell threshold is: {sell_threshold}")
      test_df = test_the_market(market, buy_threshold, sell_threshold, start, end)

      if test_df.empty != True:
        profit_or_loss = []
        buffer_stock = test_df["STOCK"].iloc[0]

        for row in range(test_df.shape[0]):
          loop_stock = test_df["STOCK"].iloc[row]

          if buffer_stock != loop_stock:
            profit_or_loss.append(test_df["TRY BALANCE"].iloc[row-1]-2000)
            buffer_stock = loop_stock

        profit_or_loss.append(test_df["TRY BALANCE"].iloc[-1]-2000)
        profit = sum(profit_or_loss)

        tries.append([buy_threshold, sell_threshold, profit])

  return pd.DataFrame(tries, columns=["BUY TH", "SELL TH", "PROFIT"])

#Variables

In [31]:
start="2023-04-01"
end="2023-06-22"
balance = 2000
#57, 63 - 64, 62 - 46, 78
buy_threshold = 64
sell_threshold = 62

#Buy or Sell Informations on Stocks

In [32]:
buy_stocks = []
sell_stocks = []
simulation_stocks = pd.DataFrame()

lookup_table = ["AKBNK.IS", "SAHOL.IS"]
portfolio = []

for stock in xu030:
  try:
    data = yf.download(stock, start=start, end=end, progress=False)
    data = create_indicators(data)
    data = score(data)

    if stock in lookup_table:
      # save_to_excel(stock, data)
      portfolio.append([data.tail(1)["Date"].item(), stock, data.tail(1)["Close"].item(), data.tail(1)["SCORE"].item(),
                        "BUY" if data["SCORE"].iloc[-1] > buy_threshold and data["SCORE"].iloc[-2] < buy_threshold else
                        "SELL" if data["SCORE"].iloc[-1] < sell_threshold and data["SCORE"].iloc[-2] > sell_threshold else
                        "SELL" if data["SCORE"].iloc[-1] < sell_threshold else "HOLD"])

    if data["SCORE"].iloc[-1] > buy_threshold and data["SCORE"].iloc[-2] < buy_threshold:
      buy_stocks.append([data.tail(1)["Date"].item(), stock, data.tail(1)["Close"].item(), data.tail(1)["SCORE"].item(), "BUY"])
      simulation_df = buy_or_sell(stock, data, balance=2000, upper_score=buy_threshold, lower_score=sell_threshold)
      simulation_stocks = pd.concat([simulation_stocks, simulation_df])

    elif data["SCORE"].iloc[-1] < sell_threshold and data["SCORE"].iloc[-2] > sell_threshold:
      sell_stocks.append([data.tail(1)["Date"].item(), stock, data.tail(1)["Close"].item(), data.tail(1)["SCORE"].item(), "SELL"])

  except:
    print("Data error at the stock named: " + stock)

In [33]:
portfolio = pd.DataFrame(portfolio, columns=["DATE","STOCK", "PRICE","SCORE", "B/S/H"])
portfolio = portfolio.sort_values(by='SCORE', ascending=False)
portfolio

,DATE,STOCK,PRICE,SCORE,B/S/H
0,2023-06-21,AKBNK.IS,19.12,67.21,HOLD
1,2023-06-21,SAHOL.IS,42.54,63.93,HOLD


In [34]:
buy_stocks = pd.DataFrame(buy_stocks, columns=["DATE","STOCK", "PRICE","SCORE", "B/S/H"])
buy_stocks = buy_stocks.sort_values(by='SCORE', ascending=False)
buy_stocks

,DATE,STOCK,PRICE,SCORE,B/S/H
1,2023-06-21,FROTO.IS,689.10,70.49,BUY
0,2023-06-21,ARCLK.IS,112.90,67.21,BUY


In [35]:
sell_stocks = pd.DataFrame(sell_stocks, columns=["DATE","STOCK", "PRICE","SCORE", "B/S/H"])
sell_stocks = sell_stocks.sort_values(by="SCORE", ascending=False)
sell_stocks

,DATE,STOCK,PRICE,SCORE,B/S/H


In [36]:
simulation_stocks

,STOCK,B/S,DATE,PRICE,STOCK BALANCE,TRY BALANCE,LAST PRICE
0,ARCLK.IS,BUY,2023-05-29,92.15,21.70,0.00,112.90
1,ARCLK.IS,SELL,2023-06-02,104.40,0.00,2265.87,112.90
2,ARCLK.IS,BUY,2023-06-15,114.20,19.84,0.00,112.90
3,ARCLK.IS,SELL,2023-06-16,111.50,0.00,2212.30,112.90
4,ARCLK.IS,BUY,2023-06-21,112.90,19.60,0.00,112.90
0,FROTO.IS,BUY,2023-05-29,557.00,3.59,0.00,689.10
1,FROTO.IS,SELL,2023-06-05,632.20,0.00,2270.02,689.10
2,FROTO.IS,BUY,2023-06-21,689.10,3.29,0.00,689.10


#Profit Calculation

In [37]:
test_df = test_the_market(xu030, buy_threshold, sell_threshold, start=start, end=end, balance=balance)

profit_or_loss = []
buffer_stock = test_df["STOCK"].iloc[0]

for row in range(test_df.shape[0]):
  loop_stock = test_df["STOCK"].iloc[row]

  if buffer_stock != loop_stock:
    profit_or_loss.append(test_df["TRY BALANCE"].iloc[row-1]-balance)
    buffer_stock = loop_stock

if test_df["B/S"].iloc[-1] == "BUY":
  profit_or_loss.append(test_df["LAST PRICE"].iloc[-1]*test_df["STOCK BALANCE"].iloc[-1]-balance)

profit = sum(profit_or_loss)
print(f"Profit is: {int(profit)} TL")

Profit is: 4985 TL


In [38]:
test_df

,STOCK,B/S,DATE,PRICE,STOCK BALANCE,TRY BALANCE,LAST PRICE
0,AKBNK.IS,BUY,2023-06-01 00:00:00,16.34,122.40,0.00,19.13
1,AKBNK.IS,SELL,2023-06-02 00:00:00,17.55,0.00,2148.10,19.13
2,AKBNK.IS,BUY,2023-06-14 00:00:00,18.37,116.94,0.00,19.13
3,AKBNK.IS,CLOSE,TODAY,19.13,0,2236.97,19.13
4,AKSEN.IS,BUY,2023-05-26 00:00:00,28.92,69.16,0.00,29.08
5,AKSEN.IS,SELL,2023-05-31 00:00:00,34.04,0.00,2354.08,29.08
6,ARCLK.IS,BUY,2023-05-29 00:00:00,92.15,21.70,0.00,113.00
7,ARCLK.IS,SELL,2023-06-02 00:00:00,104.40,0.00,2265.87,113.00
8,ARCLK.IS,BUY,2023-06-15 00:00:00,114.20,19.84,0.00,113.00
9,ARCLK.IS,SELL,2023-06-16 00:00:00,111.50,0.00,2212.30,113.00


#Finding Optimal Thresholds w/ this Buying Strategy

In [ ]:
stock_df = []
for stock in xu030:
  data = yf.download(stock, start=start, end=end, progress=False)
  data = create_indicators(data)
  data = score(data)
  data["STOCK"] = stock
  stock_df.append(data)

tries = []

for buy_threshold in range(40, 80):
  print(f"Tried buy threshold is: {buy_threshold}")
  for sell_threshold in range(40, 80):
    print(f"Tried sell threshold is: {sell_threshold}")

    test_df = pd.DataFrame()

    for df in stock_df:
      simulation_df = buy_or_sell(df["STOCK"].iloc[0], df, balance=balance, upper_score=buy_threshold, lower_score=sell_threshold)

      if simulation_df.empty != True:
        test_df = pd.concat([test_df, simulation_df])

        if test_df.empty != True:
            profit_or_loss = []
            buffer_stock = test_df["STOCK"].iloc[0]

            for row in range(test_df.shape[0]):
              loop_stock = test_df["STOCK"].iloc[row]

              if buffer_stock != loop_stock:
                profit_or_loss.append(test_df["TRY BALANCE"].iloc[row-1]-balance)
                buffer_stock = loop_stock

        test_df = test_df.reset_index().drop(["index"], axis=1)

        for row in range(test_df.shape[0]):
          if row+1 < test_df.shape[0] and test_df["B/S"].iloc[row] == "BUY" and test_df["B/S"].iloc[row+1] == "BUY" and not test_df["B/S"].iloc[row] == "CLOSE" and not test_df["B/S"].iloc[row+1] == "CLOSE":
            add = pd.DataFrame({"STOCK": [test_df["STOCK"][row]], "B/S": ["CLOSE"], "DATE": ["TODAY"], "PRICE": [test_df["LAST PRICE"][row]],
                                'STOCK BALANCE': ["0"], 'TRY BALANCE': [test_df["STOCK BALANCE"][row]*test_df["LAST PRICE"][row]], 'LAST PRICE' :[test_df["LAST PRICE"][row]]})

            df_top = test_df.loc[:row]
            df_bottom = test_df.loc[row+1:]
            test_df = pd.concat([df_top, add, df_bottom], ignore_index=True)

        profit_or_loss.append(test_df["TRY BALANCE"].iloc[-1]-balance)
        profit = sum(profit_or_loss)

        tries.append([buy_threshold, sell_threshold, profit])

  result = pd.DataFrame(tries, columns=["BUY TH", "SELL TH", "PROFIT"])
  result = result.sort_values(by="PROFIT", ascending=False)

#ML

In [148]:
dataset = pd.DataFrame()

for stock in xu100:
  print("Getting " + stock)
  data = yf.download(stock, start, end, progress=False)
  data = create_indicators(data)
  data = score(data)
  data = data.drop(["Date"], axis=1)
  data["LABEL"] = 1

  for row in range(data.shape[0]):
    if row+1 < data.shape[0] and data["Close"].iloc[row] < data["Close"].iloc[row+1]:
      data["LABEL"].iloc[row] = 2

    elif row+1 < data.shape[0] and data["Close"].iloc[row] > data["Close"].iloc[row+1]:
      data["LABEL"].iloc[row] = 0

  dataset = pd.concat([dataset, data])

<ipython-input-148-14d4e20b18c8>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["LABEL"].iloc[row] = 0
<ipython-input-148-14d4e20b18c8>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["LABEL"].iloc[row] = 2


Getting AKSA.IS


KeyboardInterrupt: ignored

In [ ]:
dataset = dataset.drop(dataset.columns[4:29], axis=1)
dataset = dataset.drop(dataset.columns[:3], axis=1)
dataset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [ ]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset):
 dataX, dataY = [], []
 for i in range(len(dataset)):
  dataX.append(dataset[i,:2])
  dataY.append(dataset[i,2])
 return np.array(dataX), np.array(dataY)

In [ ]:
trainX, trainY = create_dataset(train)
testX, testY = create_dataset(test)

In [ ]:
trainX.shape

In [ ]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [147]:
model = Sequential()
model.add(LSTM(256, input_shape=(trainX.shape[0], trainX.shape[1]), activation='relu', return_sequences=True))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(trainX, trainY, epochs=100, batch_size=64, validation_data=(testX, testY))

_, accuracy = model.evaluate(testX, testY)
print('Accuracy: %.2f' % (accuracy * 100))

Epoch 1/100


ValueError: ignored

In [ ]:

# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [ ]:
trainPredict[0]